In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
file_list = ['order_A_20230524.xlsx', 'order_B_20230524.xlsx','order_C_20230524.xlsx', 'order_D_20230524.xlsx']
# 結合後のデータを格納するための空のデータフレームを作成
df_order = pd.DataFrame()

# ファイルを一つずつ処理する
for file_name in file_list:
    # Excelファイルを読み込む
    temp_df = pd.read_excel(file_name, header=None).T

    # データフレームの列名を指定
    temp_df.columns = ['品名', '個数']

    # concat()でデータフレームを連結
    df_order = pd.concat([df_order, temp_df], ignore_index=True)

# print(df_order)
# 品名ごとに個数を合計
summary = df_order.groupby('品名')['個数'].sum().reset_index()
print(summary)

      品名  個数
0  ほうれん草  23
1   キャベツ  21
2    トマト  31
3   ニンジン  32
4    レタス  42
5     大根  15
6     白菜  25


In [ ]:
# 在庫を読み込む
df_inventory = pd.read_excel('inventory.xlsx')
print(df_inventory)

# 最新の在庫状況（-1で最後の1行)
latest_inventory = df_inventory.iloc[-1]
print(latest_inventory)


           日付   曜日  トマト  キャベツ  レタス  白菜  ほうれん草  大根  ニンジン
0  2023-05-01  Mon  106    70  101  64     95  65    68
1  2023-05-02  Tue   75    71   78  47     60  45    69
2  2023-05-03  Wed   61    81   82  53     51  25    89
3  2023-05-04  Thu   47    49   93  38     75  41    91
4  2023-05-05  Fri   67    52   67  21     97  28    56
5  2023-05-06  Sat   43    92   96  40     66  50    37
6  2023-05-07  Sun   81    39   74  18    104  38    77
7  2023-05-08  Mon  104    83   29  33     72  56    55
8  2023-05-09  Tue   31    36   84  35     92  44    78
9  2023-05-10  Wed   92    75   32  29     43  22    62
10 2023-05-11  Thu   61    29   54  52     33  78    92
11 2023-05-12  Fri   55    40   60  15     27  43    29
12 2023-05-13  Sat   49   103   41  10     35  33    58
13 2023-05-14  Sun   91    73  103  84     75  48    50
日付       2023-05-14 00:00:00
曜日                       Sun
トマト                       91
キャベツ                      73
レタス                      103
白菜             

In [ ]:
df_pickup = pd.read_excel('pickup.xlsx',index_col=0)
print(df_pickup)

      トマト  キャベツ  レタス  白菜  ほうれん草  大根  ニンジン
しきい値   50    40   50  30     40  30    40
追加量   100    80  100  60     80  60    80


In [ ]:
shikii = df_pickup.loc["しきい値"]
print(shikii)

トマト      50
キャベツ     40
レタス      50
白菜       30
ほうれん草    40
大根       30
ニンジン     40
Name: しきい値, dtype: int64


In [ ]:
def calc(row):
  order = latest_inventory[row["品名"]] - row['個数']
  return order

In [ ]:
summary["注文後在庫"] = summary.apply(calc, axis=1)
print(summary)

      品名  個数  注文後在庫
0  ほうれん草  23     52
1   キャベツ  21     52
2    トマト  31     60
3   ニンジン  32     18
4    レタス  42     61
5     大根  15     33
6     白菜  25     59


In [ ]:
def pickup(row):
  return row['注文後在庫'] - shikii[row['品名']]

In [ ]:
summary["注文"] = summary.apply(pickup, axis=1)
print(summary)

      品名  個数  注文後在庫  注文
0  ほうれん草  23     52  12
1   キャベツ  21     52  12
2    トマト  31     60  10
3   ニンジン  32     18 -22
4    レタス  42     61  11
5     大根  15     33   3
6     白菜  25     59  29


In [ ]:
chumon = summary[summary["注文"] < 0]
print(chumon)


     品名  個数  注文後在庫  注文
3  ニンジン  32     18 -22


In [ ]:
# 品名をキー、注文を値とした辞書を作成
order_dict = summary.set_index("品名")["注文"].to_dict()
print(order_dict)

{'ほうれん草': 12, 'キャベツ': 12, 'トマト': 10, 'ニンジン': -22, 'レタス': 11, '大根': 3, '白菜': 29}


In [ ]:
import datetime
new_inventory = pd.Series(
    {**{'日付': datetime.datetime.now(), '曜日': datetime.datetime.now().strftime('%a')}, **order_dict},
    name=len(df_inventory)
)

df_inventory = pd.concat([df_inventory, new_inventory.to_frame().T], ignore_index=True)

df_inventory

,日付,曜日,トマト,キャベツ,レタス,白菜,ほうれん草,大根,ニンジン
0,2023-05-01 00:00:00,Mon,106,70,101,64,95,65,68
1,2023-05-02 00:00:00,Tue,75,71,78,47,60,45,69
2,2023-05-03 00:00:00,Wed,61,81,82,53,51,25,89
3,2023-05-04 00:00:00,Thu,47,49,93,38,75,41,91
4,2023-05-05 00:00:00,Fri,67,52,67,21,97,28,56
5,2023-05-06 00:00:00,Sat,43,92,96,40,66,50,37
6,2023-05-07 00:00:00,Sun,81,39,74,18,104,38,77
7,2023-05-08 00:00:00,Mon,104,83,29,33,72,56,55
8,2023-05-09 00:00:00,Tue,31,36,84,35,92,44,78
9,2023-05-10 00:00:00,Wed,92,75,32,29,43,22,62


In [ ]:
# 更新されたデータフレームを元のExcelファイルに書き込む
df_inventory.to_excel('/content/inventory.xlsx', index=False)

In [ ]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


# ニンジンの注文がマイナスのときだけ送信
target_item = summary[(summary['品名'] == 'ニンジン') & (summary['注文'].astype(float) < 0)]

# Looking to send emails in production? Check out our Email API/SMTP product!
import smtplib

item_name = target_item.iloc[0]['品名']
order_qty = target_item.iloc[0]['注文']

sender = "Private Person <from@example.com>"
receiver = "A Test User <to@example.com>"

msg = MIMEMultipart()
msg['Subject'] = f"【追加注文のお願い】{item_name}の注文数不足について"
msg['From'] = sender
msg['To'] = receiver

body = f"""
お疲れ様です。

現在の注文データを確認したところ、「{item_name}」の注文数が {order_qty} となっております。
不足分について、至急追加注文の手配をお願いいたします。

ご対応のほど、よろしくお願いいたします。
"""

msg.attach(MIMEText(body, 'plain', 'utf-8'))

with smtplib.SMTP("sandbox.smtp.mailtrap.io", 2525) as server:
    server.starttls()
    server.login("b28a9ad0ab22c9", "2898698d4e7969")
    server.sendmail(sender, receiver, msg.as_string())